# s3prlvc
[![Generic badge](https://img.shields.io/badge/GitHub-s3prlvc-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]

Author: [tarepan]

[github]:https://github.com/tarepan/S3PRL_VC
[notebook]:https://colab.research.google.com/github/tarepan/S3PRL_VC/blob/main/s3prlvc.ipynb
[tarepan]:https://github.com/tarepan

## Colab Check
Check
- Google Colaboratory runnning time
- GPU type
- Python version
- CUDA version

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
!head -n 1 /proc/driver/nvidia/gpus/**/information
!python --version
!pip show torch | sed '2!d'
!/usr/local/cuda/bin/nvcc --version | sed '4!d'

## Setup

Install the package from `tarepan/S3PRL_VC` public repository

In [ ]:
# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

# Dedicated dependencies install
# !pip install "torch==1.10.0" -q      # Based on your PyTorch environment
# !pip install "torchaudio==0.10.0" -q # Based on your PyTorch environment

# Prerequisites
!apt-get install sox

# repository install
!pip uninstall s3prlvc -y -q
!pip install git+https://github.com/tarepan/S3PRL_VC -q

# Pretrained models
## Unit2Mel: vq-wav2vec (If you omit this 2 lines, s3prl automatically download the model)
!mkdir -p /root/.cache/torch/hub
!cp -r /content/gdrive/MyDrive/ML_data/s3prl_cache /root/.cache/torch/hub
## Mel2Wave: HiFi-GAN (prerequisite: download the model from `https://drive.google.com/file/d/12w1LpF6HjsJBmOUUkS6LV1d7AX18SA7u`)
!mkdir -p /root/.cache/parallel_wavegan/12w1LpF6HjsJBmOUUkS6LV1d7AX18SA7u
!cp -r /content/gdrive/MyDrive/ML_data/pretrained/HiFiGAN/hifigan_vctk/* /root/.cache/parallel_wavegan/12w1LpF6HjsJBmOUUkS6LV1d7AX18SA7u

## Training

In [ ]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir gdrive/MyDrive/ML_results/s3prlvc

# Train
!python -m s3prlvc.main_train \
    train.ckpt_log.dir_root=gdrive/MyDrive/ML_results/s3prlvc \
    train.ckpt_log.name_exp=default \
    train.ckpt_log.name_version=version_default \
    data.adress_data_root=gdrive/MyDrive/ML_data \


## Inference

In [ ]:
# .wav I/O
!python -m s3prlvc.main_inference \
    model_ckpt_path="gdrive/MyDrive/ML_results/s3prlvc/default/version_-1/checkpoints/last.ckpt" \
    i_wav_path="./hello.wav"

In [ ]:
# # data I/O
# from IPython.display import Audio, display

# from torch import inference_mode
# import torchaudio
# import soundfile as sf

# from rnnms.model import RNNMS


# torchaudio.set_audio_backend("sox_io")
# CKPT_PATH = "gdrive/MyDrive/ML_results/rnnms/default/version_-1/checkpoints/last.ckpt"
# model = RNNMS.load_from_checkpoint(checkpoint_path=CKPT_PATH)

# wave, orig_sr = model.sample_wave()
# # import librosa
# # I_WAV_PATH = ""
# # wave, orig_sr = librosa.load(I_WAV_PATH)

# with inference_mode():
#     mel = model.wav2mel(wave, orig_sr)
#     o_wave, o_sr = model.predict(mel)
# o_wave = o_wave[0].to('cpu').detach().numpy()

# display(Audio(o_wave, rate=o_sr))

In [ ]:
# # Usage stat
# ## GPU
# !nvidia-smi -l 3
# ## CPU
# !vmstat 5
# !top